In [1]:
from pathlib import Path
import shutil
from google.colab import drive

drive.mount('/content/drive')

BASE_DIR = Path("/content/drive/MyDrive/preprocessing")

ROOT_NAMES = [
    "Holistic_hands_frames",
    "Holistic_body_frames",
    "Holistic_full_frames",
    "Keypoints_json",
]

for name in ROOT_NAMES:
    root = BASE_DIR / name
    root.mkdir(parents=True, exist_ok=True)   # 루트 폴더 없으면 생성
    print(f"[ROOT] {root}")

    for pid in range(1, 11):
        d = root / str(pid)

        # 폴더가 이미 있으면 안의 내용만 싹 비우고 다시 생성
        if d.exists():
            shutil.rmtree(d)

        d.mkdir(parents=True, exist_ok=True)
        print(f"  -> prepared: {d}")

print("✅ 모든 root에 1~10 폴더 생성 / 초기화 완료")

Mounted at /content/drive
[ROOT] /content/drive/MyDrive/preprocessing/Holistic_hands_frames
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/1
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/2
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/3
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/4
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/5
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/6
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/7
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/8
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/9
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_hands_frames/10
[ROOT] /content/drive/MyDrive/preprocessing/Holistic_body_frames
  -> prepared: /content/drive/MyDrive/preprocessing/Holistic_body_frames/1
  -> pr

In [2]:
# ============================================
# 1) 기본 세팅 & Mediapipe 설치/로드
# ============================================
from google.colab import drive
drive.mount('/content/drive')

!pip -q install mediapipe==0.10.14


from pathlib import Path
import os, cv2, json
import numpy as np
import mediapipe as mp
from tqdm.auto import tqdm

# 🔧 경로 설정 (네 구조 기준)
BASE_DIR      = Path("/content/drive/MyDrive/preprocessing")
CROPPED_DIR   = BASE_DIR / "cropped_videos"        # 22개 단어 × 10명 × 5각도
HANDS_ROOT    = BASE_DIR / "Holistic_hands_frames"
BODY_ROOT     = BASE_DIR / "Holistic_body_frames"
FULL_ROOT     = BASE_DIR / "Holistic_full_frames"
JSON_ROOT     = BASE_DIR / "Keypoints_json"

# 사람 ID 매핑: P01 → "1", ..., P10 → "10"
P_TO_ID = {f"P{idx:02d}": str(idx) for idx in range(1, 11)}

IMG_SIZE   = 720   # 프레임 캔버스 크기 (정사각, 예전 코드랑 비슷하게 크게)
FRAME_STRIDE = 1   # 모든 프레임 사용 (필요하면 2로 줄여서 속도↑)

# Mediapipe 모듈
mp_holistic = mp.solutions.holistic
mp_hands    = mp.solutions.hands
mp_pose     = mp.solutions.pose
mp_fmesh    = mp.solutions.face_mesh

HAND_CONNS = list(mp_hands.HAND_CONNECTIONS)
POSE_CONNS = list(mp_pose.POSE_CONNECTIONS)
FACEMESH_CONTOURS = list(mp_fmesh.FACEMESH_CONTOURS)   # 얼굴 윤곽선만 사용

# === 머리(얼굴) 없는 바디용 설정 ===
# Mediapipe Pose index 기준: 0~10이 얼굴/머리 쪽 (nose, eye, ear, mouth 등)
HEAD_IDS = list(range(0, 11))  # 0~10

# 몸통/팔/다리 위주로 쓸 landmark id들 (어깨~발, 발끝)
BODY_ONLY_IDS = {
    11, 12, 13, 14, 15, 16,      # 어깨/팔
    23, 24, 25, 26, 27, 28,      # 엉덩이/무릎/발목
    31, 32                       # 발끝
}

POSE_CONNS_BODY = [
    (a, b)
    for (a, b) in POSE_CONNS
    if a in BODY_ONLY_IDS and b in BODY_ONLY_IDS
]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [3]:
# ============================================
# 2) 키포인트 변환 & 그리기 헬퍼
# ============================================

def lm_to_arrays(lm_list, n_expected):
    """Mediapipe landmark 리스트 → (x,y,z,vis) numpy 배열 (길이 n_expected, 없으면 NaN/0)"""
    if lm_list is None:
        xs = np.full(n_expected, np.nan)
        ys = np.full(n_expected, np.nan)
        zs = np.full(n_expected, np.nan)
        vs = np.zeros(n_expected)
        return xs, ys, zs, vs

    xs = np.array([p.x for p in lm_list.landmark])
    ys = np.array([p.y for p in lm_list.landmark])
    zs = np.array([getattr(p, "z", 0.0) for p in lm_list.landmark])
    vs = np.ones(len(xs))

    # 길이 보정 (부족하면 패딩, 많으면 자르기)
    if len(xs) < n_expected:
        pad = n_expected - len(xs)
        xs = np.pad(xs, (0, pad), constant_values=np.nan)
        ys = np.pad(ys, (0, pad), constant_values=np.nan)
        zs = np.pad(zs, (0, pad), constant_values=np.nan)
        vs = np.pad(vs, (0, pad), constant_values=0.0)
    elif len(xs) > n_expected:
        xs, ys, zs, vs = xs[:n_expected], ys[:n_expected], zs[:n_expected], vs[:n_expected]

    return xs, ys, zs, vs


def arrays_to_json(xs, ys, zs, vs):
    """(x,y,z,vis) 배열 → JSON-friendly 리스트 (NaN은 null로)"""
    out = []
    for x, y, z, v in zip(xs, ys, zs, vs):
        if not np.isfinite(x) or not np.isfinite(y):
            out.append({"x": None, "y": None, "z": None, "v": float(v)})
        else:
            out.append({"x": float(x), "y": float(y), "z": float(z), "v": float(v)})
    return out


def draw_points_and_lines(canvas, xs, ys, vs, conns, color_line, color_dot,
                          r=2, thick=1, size=IMG_SIZE):
    """정규화(0~1) 좌표를 캔버스 위에 스켈레톤으로 그림"""
    def to_px(v): return int(v * size)

    pts = [
        (to_px(xs[i]), to_px(ys[i]))
        if np.isfinite(xs[i]) and np.isfinite(ys[i]) else None
        for i in range(len(xs))
    ]

    # 선 그리기
    for a, b in conns:
        if a < len(pts) and b < len(pts) and pts[a] and pts[b] and vs[a] > 0.5 and vs[b] > 0.5:
            cv2.line(canvas, pts[a], pts[b], color_line, thick, cv2.LINE_AA)

    # 점 그리기
    for i, p in enumerate(pts):
        if p and vs[i] > 0.5:
            cv2.circle(canvas, p, r, color_dot, -1, cv2.LINE_AA)

In [ ]:
from pathlib import Path
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

CROPPED_DIR = Path("/content/drive/MyDrive/preprocessing/cropped_videos")

video_paths = sorted(CROPPED_DIR.rglob("*.mp4"))
print("총 mp4 개수:", len(video_paths))

# 어떤 단어 폴더들이 실제로 mp4를 가지고 있는지 확인
word_names = [p.parent.parent.name for p in video_paths]
print("단어 폴더 종류:", sorted(set(word_names)))
print()

cnt = Counter(word_names)
for w, c in sorted(cnt.items()):
    print(f"{w}: {c}개")

In [4]:
# ============================================
# 3) 모든 크롭 영상 처리 (영상당 16프레임 고정)
#    - hands / body / full 이미지
#    - hands / body / full JSON
# ============================================

N_FRAMES = 16  # 한 영상당 만들고 싶은 프레임 개수

video_paths = sorted(CROPPED_DIR.rglob("*.mp4"))
print("총 비디오 개수:", len(video_paths))

with mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    refine_face_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
) as holistic:

    for vpath in tqdm(video_paths):
        # 예: .../cropped_videos/WORD0029_검사/P01/xxxx.mp4
        word_folder   = vpath.parent.parent.name
        person_folder = vpath.parent.name
        file_stem     = vpath.stem

        person_id = P_TO_ID.get(person_folder)
        if person_id is None:
            print(f"[WARN] person id not found for {vpath}")
            continue

        cap = cv2.VideoCapture(str(vpath))
        if not cap.isOpened():
            print(f"[WARN] 열 수 없음: {vpath}")
            continue

        # 이 영상에서 "유효한" 프레임들을 모아둘 리스트
        valid_frames = []  # 각 원소: dict(... 아래 참고 ...)
        frame_idx = 0

        while True:
            ok, frame = cap.read()
            if not ok:
                break

            # 필요하면 FRAME_STRIDE로 속도 조절 (지금은 1 → 모든 프레임)
            if frame_idx % FRAME_STRIDE != 0:
                frame_idx += 1
                continue

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            res = holistic.process(rgb)

            # ---- 키포인트 numpy 배열 생성 ----
            fx, fy, fz, fv = lm_to_arrays(res.face_landmarks,      468)
            px, py, pz, pv = lm_to_arrays(res.pose_landmarks,       33)
            xL, yL, zL, vL = lm_to_arrays(res.left_hand_landmarks,  21)
            xR, yR, zR, vR = lm_to_arrays(res.right_hand_landmarks, 21)

            # 이 프레임에 아무 키포인트도 없으면 스킵
            has_any = (
                np.any(fv > 0.5) or
                np.any(pv > 0.5) or
                np.any(vL > 0.5) or
                np.any(vR > 0.5)
            )
            if not has_any:
                frame_idx += 1
                continue

            # ---- 캔버스 3개 준비 ----
            hands_img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
            body_img  = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
            full_img  = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)

            # ========= ① hands: 손만 =========
            draw_points_and_lines(hands_img, xL, yL, vL, HAND_CONNS,
                                  (255,255,255), (255,255,255), r=2, thick=1)
            draw_points_and_lines(hands_img, xR, yR, vR, HAND_CONNS,
                                  (255,255,255), (255,255,255), r=2, thick=1)

            # ========= ② body: 얼굴 없는 몸+손 =========
            pv_body = pv.copy()
            pv_body[HEAD_IDS] = 0.0
            draw_points_and_lines(
                body_img,
                px, py, pv_body,
                POSE_CONNS_BODY,
                (255,180,80), (255,220,120),
                r=3, thick=3
            )
            draw_points_and_lines(body_img, xL, yL, vL, HAND_CONNS,
                                  (255,255,255), (255,255,255), r=2, thick=2)
            draw_points_and_lines(body_img, xR, yR, vR, HAND_CONNS,
                                  (255,255,255), (255,255,255), r=2, thick=2)

            # ========= ③ full: 얼굴+몸+손 =========
            draw_points_and_lines(full_img, fx, fy, fv, FACEMESH_CONTOURS,
                                  (200,200,200), (180,180,180), r=1, thick=1)
            draw_points_and_lines(full_img, px, py, pv, POSE_CONNS,
                                  (255,180,80), (255,220,120), r=3, thick=3)
            draw_points_and_lines(full_img, xL, yL, vL, HAND_CONNS,
                                  (255,255,255), (255,255,255), r=2, thick=2)
            draw_points_and_lines(full_img, xR, yR, vR, HAND_CONNS,
                                  (255,255,255), (255,255,255), r=2, thick=2)

            # JSON 좌표 미리 만들어 두기
            face_json = arrays_to_json(fx, fy, fz, fv)
            pose_json = arrays_to_json(px, py, pz, pv)
            left_json = arrays_to_json(xL, yL, zL, vL)
            right_json = arrays_to_json(xR, yR, zR, vR)

            # 이 프레임 정보를 리스트에 저장 (바로 파일로 안 쓰고)
            valid_frames.append({
                "orig_frame_index": int(frame_idx),
                "hands_img": hands_img,
                "body_img": body_img,
                "full_img": full_img,
                "face_json": face_json,
                "pose_json": pose_json,
                "left_json": left_json,
                "right_json": right_json,
            })

            frame_idx += 1

        cap.release()

        # 이 영상에서 유효 프레임이 하나도 없으면 패스
        n_valid = len(valid_frames)
        if n_valid == 0:
            print(f"[INFO] no valid landmarks in video: {vpath}")
            continue

        # ---- 여기서 N_FRAMES(16)개 인덱스 선택 ----
        if n_valid >= N_FRAMES:
            # 0 ~ n_valid-1 범위에서 균등하게 16개 뽑기
            chosen_idx = np.linspace(0, n_valid - 1, N_FRAMES, dtype=int)
        else:
            # 유효 프레임이 16개 미만이면 마지막 프레임을 반복해서 채우기
            base = np.arange(n_valid)
            pad  = np.full(N_FRAMES - n_valid, n_valid - 1)
            chosen_idx = np.concatenate([base, pad])

        # 출력 디렉토리 준비
        out_hands_dir = HANDS_ROOT / person_id
        out_body_dir  = BODY_ROOT / person_id
        out_full_dir  = FULL_ROOT / person_id
        json_dir      = JSON_ROOT / person_id

        out_hands_dir.mkdir(parents=True, exist_ok=True)
        out_body_dir.mkdir(parents=True, exist_ok=True)
        out_full_dir.mkdir(parents=True, exist_ok=True)
        json_dir.mkdir(parents=True, exist_ok=True)

        # 선택된 16개 프레임만 실제 파일로 저장
        for seq_i, idx in enumerate(chosen_idx):
            item = valid_frames[idx]
            orig_fidx = item["orig_frame_index"]

            # s00 ~ s15 : 시퀀스 내 위치
            base_name = f"{word_folder}_{file_stem}_s{seq_i:02d}"

            # PNG 저장
            cv2.imwrite(str(out_hands_dir / f"{base_name}.png"), item["hands_img"])
            cv2.imwrite(str(out_body_dir  / f"{base_name}.png"), item["body_img"])
            cv2.imwrite(str(out_full_dir  / f"{base_name}.png"), item["full_img"])

            # 공통 메타
            common_meta = {
                "video_path": str(vpath),
                "word_folder": word_folder,
                "person_folder": person_folder,
                "person_id": person_id,
                "file_stem": file_stem,
                "frame_index": int(orig_fidx),  # 원래 프레임 번호
                "seq_index": int(seq_i),        # 시퀀스 내 인덱스(0~15)
                "img_size": int(IMG_SIZE),
            }

            # hands JSON
            obj_hands = {
                **common_meta,
                "mode": "hands",
                "face": [],
                "pose": [],
                "left_hand": item["left_json"],
                "right_hand": item["right_json"],
            }
            with open(json_dir / f"{base_name}_hands.json", "w", encoding="utf-8") as f:
                json.dump(obj_hands, f, ensure_ascii=False)

            # body JSON
            obj_body = {
                **common_meta,
                "mode": "body",
                "face": [],
                "pose": item["pose_json"],
                "left_hand": item["left_json"],
                "right_hand": item["right_json"],
            }
            with open(json_dir / f"{base_name}_body.json", "w", encoding="utf-8") as f:
                json.dump(obj_body, f, ensure_ascii=False)

            # full JSON
            obj_full = {
                **common_meta,
                "mode": "full",
                "face": item["face_json"],
                "pose": item["pose_json"],
                "left_hand": item["left_json"],
                "right_hand": item["right_json"],
            }
            with open(json_dir / f"{base_name}_full.json", "w", encoding="utf-8") as f:
                json.dump(obj_full, f, ensure_ascii=False)

print("\n✅ 모든 크롭 영상에 대해 '길이 16' 시퀀스로 hands/body/full 이미지 + JSON 생성")

총 비디오 개수: 1097


  0%|          | 0/1097 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



✅ 모든 크롭 영상에 대해 '길이 16' 시퀀스로 hands/body/full 이미지 + JSON 생성
